<a href="https://colab.research.google.com/github/s528661/nlp100/blob/main/nlp100_9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 80. ID番号への変換Permalink

問題51で構築した学習データ中の単語にユニークなID番号を付与したい．学習データ中で最も頻出する単語に1，2番目に頻出する単語に2，……といった方法で，学習データ中で2回以上出現する単語にID番号を付与せよ．そして，与えられた単語列に対して，ID番号の列を返す関数を実装せよ．ただし，出現頻度が2回未満の単語のID番号はすべて0とせよ．

In [1]:
!pip install mecab-python3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 581.7/581.7 kB 6.3 MB/s eta 0:00:00


In [2]:
import re,sys,os

import pickle
import time
import copy

import math
import random
import string

from collections import defaultdict, Counter

import numpy as np
import pandas as pd
import scipy as scp

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from torch.utils.data import Dataset, DataLoader, TensorDataset

from sklearn import feature_extraction, preprocessing

from sklearn import svm
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression, Ridge, Lasso, LogisticRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, RandomForestClassifier
from sklearn.neighbors import KNeighborsRegressor

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold, StratifiedKFold, GroupKFold
from sklearn.model_selection import cross_val_score

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, mean_absolute_percentage_error, f1_score
from sklearn.metrics import accuracy_score,  precision_score, recall_score, confusion_matrix, roc_auc_score

# import dask.array as da

import lightgbm as lgb

# Neural Language Processing
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.feature_selection import SelectKBest

from sklearn.linear_model import RidgeClassifier
import MeCab

import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

In [3]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/00359/NewsAggregatorDataset.zip --no-check-certificate
!unzip ./NewsAggregatorDataset.zip

--2023-12-31 09:13:08--  https://archive.ics.uci.edu/ml/machine-learning-databases/00359/NewsAggregatorDataset.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified
Saving to: ‘NewsAggregatorDataset.zip’

NewsAggregatorDatas     [     <=>            ]  27.87M  28.1MB/s    in 1.0s    

2023-12-31 09:13:09 (28.1 MB/s) - ‘NewsAggregatorDataset.zip’ saved [29224203]

Archive:  ./NewsAggregatorDataset.zip
  inflating: 2pageSessions.csv       
   creating: __MACOSX/
  inflating: __MACOSX/._2pageSessions.csv  
  inflating: newsCorpora.csv         
  inflating: __MACOSX/._newsCorpora.csv  
  inflating: readme.txt              
  inflating: __MACOSX/._readme.txt   


In [4]:
data = pd.read_csv('newsCorpora.csv',delimiter='\t',header=None,
                 names=["ID", "TITLE", "URL", "PUBLISHER", "CATEGORY", "STORY", "HOSTNAME", "TIMESTAMP"])

In [5]:
fix_data = data[data['PUBLISHER'].isin(["Reuters", "Huffington Post", "Businessweek", "Contactmusic.com", "Daily Mail"])]

In [6]:
fix_data

,ID,TITLE,URL,PUBLISHER,CATEGORY,STORY,HOSTNAME,TIMESTAMP
12,13,Europe reaches crunch point on banking union,http://in.reuters.com/article/2014/03/10/eu-ba...,Reuters,b,dPhGU51DcrolUIMxbRm0InaHGA2XM,in.reuters.com,1394470501755
13,14,ECB FOCUS-Stronger euro drowns out ECB's messa...,http://in.reuters.com/article/2014/03/10/ecb-p...,Reuters,b,dPhGU51DcrolUIMxbRm0InaHGA2XM,in.reuters.com,1394470501948
19,20,"Euro Anxieties Wane as Bunds Top Treasuries, S...",http://www.businessweek.com/news/2014-03-10/ge...,Businessweek,b,dPhGU51DcrolUIMxbRm0InaHGA2XM,www.businessweek.com,1394470503148
20,21,Noyer Says Strong Euro Creates Unwarranted Eco...,http://www.businessweek.com/news/2014-03-10/no...,Businessweek,b,dPhGU51DcrolUIMxbRm0InaHGA2XM,www.businessweek.com,1394470503366
29,30,REFILE-Bad loan triggers key feature in ECB ba...,http://in.reuters.com/article/2014/03/10/euroz...,Reuters,b,dPhGU51DcrolUIMxbRm0InaHGA2XM,in.reuters.com,1394470505070
...,...,...,...,...,...,...,...,...
422300,422819,UN: Ebola Could Eventually Infect 20000 People,http://www.huffingtonpost.com/2014/08/28/ebola...,Huffington Post,m,dhhnSHVoyA7ENBM3boDX_D_-3PV6M,www.huffingtonpost.com,1409228423333
422301,422820,"Ebola toll tops 1550, continues to accelerate ...",http://in.reuters.com/article/2014/08/28/healt...,Reuters,m,dhhnSHVoyA7ENBM3boDX_D_-3PV6M,in.reuters.com,1409228423827
422302,422821,"UPDATE 1-Ebola toll tops 1550, outbreak accele...",http://in.reuters.com/article/2014/08/28/healt...,Reuters,m,dhhnSHVoyA7ENBM3boDX_D_-3PV6M,in.reuters.com,1409228424525
422309,422828,"Ebola Cases May Surpass 20000, WHO Says in Upd...",http://www.businessweek.com/news/2014-08-28/eb...,Businessweek,m,dhhnSHVoyA7ENBM3boDX_D_-3PV6M,www.businessweek.com,1409228426943


In [7]:
def preprocessing(text):
  text = "".join([i for i in text if i not in string.punctuation])
  text = text.lower()
  text = re.sub("[0-9]+", "", text)
  return text

In [8]:
fix_data['pTITLE'] = fix_data['TITLE'].apply(lambda x: preprocessing(x))

<ipython-input-8-0330034268ea>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fix_data['pTITLE'] = fix_data['TITLE'].apply(lambda x: preprocessing(x))


In [9]:
fix_data

,ID,TITLE,URL,PUBLISHER,CATEGORY,STORY,HOSTNAME,TIMESTAMP,pTITLE
12,13,Europe reaches crunch point on banking union,http://in.reuters.com/article/2014/03/10/eu-ba...,Reuters,b,dPhGU51DcrolUIMxbRm0InaHGA2XM,in.reuters.com,1394470501755,europe reaches crunch point on banking union
13,14,ECB FOCUS-Stronger euro drowns out ECB's messa...,http://in.reuters.com/article/2014/03/10/ecb-p...,Reuters,b,dPhGU51DcrolUIMxbRm0InaHGA2XM,in.reuters.com,1394470501948,ecb focusstronger euro drowns out ecbs message...
19,20,"Euro Anxieties Wane as Bunds Top Treasuries, S...",http://www.businessweek.com/news/2014-03-10/ge...,Businessweek,b,dPhGU51DcrolUIMxbRm0InaHGA2XM,www.businessweek.com,1394470503148,euro anxieties wane as bunds top treasuries sp...
20,21,Noyer Says Strong Euro Creates Unwarranted Eco...,http://www.businessweek.com/news/2014-03-10/no...,Businessweek,b,dPhGU51DcrolUIMxbRm0InaHGA2XM,www.businessweek.com,1394470503366,noyer says strong euro creates unwarranted eco...
29,30,REFILE-Bad loan triggers key feature in ECB ba...,http://in.reuters.com/article/2014/03/10/euroz...,Reuters,b,dPhGU51DcrolUIMxbRm0InaHGA2XM,in.reuters.com,1394470505070,refilebad loan triggers key feature in ecb ban...
...,...,...,...,...,...,...,...,...,...
422300,422819,UN: Ebola Could Eventually Infect 20000 People,http://www.huffingtonpost.com/2014/08/28/ebola...,Huffington Post,m,dhhnSHVoyA7ENBM3boDX_D_-3PV6M,www.huffingtonpost.com,1409228423333,un ebola could eventually infect people
422301,422820,"Ebola toll tops 1550, continues to accelerate ...",http://in.reuters.com/article/2014/08/28/healt...,Reuters,m,dhhnSHVoyA7ENBM3boDX_D_-3PV6M,in.reuters.com,1409228423827,ebola toll tops continues to accelerate who
422302,422821,"UPDATE 1-Ebola toll tops 1550, outbreak accele...",http://in.reuters.com/article/2014/08/28/healt...,Reuters,m,dhhnSHVoyA7ENBM3boDX_D_-3PV6M,in.reuters.com,1409228424525,update ebola toll tops outbreak accelerates who
422309,422828,"Ebola Cases May Surpass 20000, WHO Says in Upd...",http://www.businessweek.com/news/2014-08-28/eb...,Businessweek,m,dhhnSHVoyA7ENBM3boDX_D_-3PV6M,www.businessweek.com,1409228426943,ebola cases may surpass who says in updated plan


In [10]:
train_val, test = train_test_split(fix_data, test_size=0.2, shuffle=True)
train, val = train_test_split(train_val, test_size=0.1, shuffle=True)
train = train.reset_index(drop=True)
val = val.reset_index(drop=True)
test = test.reset_index(drop=True)

In [11]:
freq_words = defaultdict(int)
for title in train['pTITLE']:
  for word in title.split():
    freq_words[word] += 1

In [12]:
freq_words = sorted(freq_words.items(), key=lambda x:x[1], reverse=True)

In [13]:
db_words = []
for id, (word, count) in enumerate(freq_words):
  if count < 2:
    db_words.append([0,word])
  else:
    db_words.append([id+1,word])

# 81. RNNによる予測

In [14]:
class RNN(nn.Module):
  def __init__(self,dw,dh,n_word,n_label):
    super().__init__()
    self.rnn = nn.RNN(dw,dh,nonlinearity="tanh")
    self.emb = nn.Embedding(n_word,dw)
    self.layer = nn.Linear(dh,n_label,bias=True)
    self.softmax = nn.Softmax()

  def forward(self,x,h):
    x = self.emb(x)
    x, h = self.rnn(x,h=None)
    x = x[:,-1,:]
    x = self.softmax(x)
    return x, h

# 82. 確率的勾配降下法による学習  
確率的勾配降下法（SGD: Stochastic Gradient Descent）を用いて，問題81で構築したモデルを学習せよ．訓練データ上の損失と正解率，評価データ上の損失と正解率を表示しながらモデルを学習し，適当な基準（例えば10エポックなど）で終了させよ．

In [15]:
labels = list(set(fix_data['CATEGORY']))

In [16]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

In [17]:
def one_hot_encoding(series):
    le = LabelEncoder()
    lvl = le.fit_transform(series)

    ohe = OneHotEncoder()
    enced = ohe.fit_transform(lvl.reshape(-1,1)).astype(int)
    names = [(series.name+"_")+str(s) for s in le.classes_]
    r_df = pd.DataFrame(index=series.index, columns=names, data=enced.toarray(), dtype=np.int64)
    return r_df

In [18]:
train_label01 = one_hot_encoding(train['CATEGORY'])
val_label01 = one_hot_encoding(val['CATEGORY'])
test_label01 = one_hot_encoding(test['CATEGORY'])

In [19]:
#train = pd.concat([train,train_label01],axis=1)
#val = pd.concat([val,val_label01],axis=1)
#test = pd.concat([test,test_label01],axis=1)

In [20]:
def data2vec(data, X_name, y_name):
  vectorizer = TfidfVectorizer()
  X = vectorizer.fit_transform(data[X_name]).toarray()
  y = one_hot_encoding(data[y_name]).values
  #Xy = pd.concat([X,y], axis=1)
  #Xy = pd.DataFrame(Xy, columns=vectorizer.get_feature_names_out())
  tX = torch.tensor(X, dtype=torch.int64)
  ty = torch.tensor(y, dtype=torch.int64)
  return tX, ty

In [21]:
X_train, y_train = data2vec(train,'pTITLE','CATEGORY')
X_val, y_val = data2vec(val,'pTITLE','CATEGORY')
X_test, y_test = data2vec(test,'pTITLE','CATEGORY')

In [31]:
train_loader = {'X':X_train, 'y':y_train}
val_loader = {'X':X_val, 'y':y_val}
test_loader = {'X':X_test, 'y':y_test}

In [22]:
dw = 300
dh = 50
n_word = len(db_words) + 1
n_label = len(labels)

In [23]:
model = RNN(dw,dh,n_word,n_label)

In [24]:
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-1)

In [37]:
batch_size = 16
n_epochs = 10

In [27]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [38]:
def train(d_train, d_val, batch_size, model, criterion, optimizer, n_epochs, device, collate_fn=None):

  model.to(device)

  dl_train = DataLoader(d_train, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
  dl_val = DataLoader(d_val, batch_size=1, shuffle=False)

  scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, n_epochs, eta_min=1e-5, last_epoch=-1)

  log_train = []
  log_val = []
  for epoch in range(n_epochs):
    model.train()
    for data in dl_train:
      optimizer.zero_grad()
      X_train = data['X']
      y_train = data['y']

      y__train_pred = model(X_train)

      train_loss = criterion(y__train_pred, y_train)

      train_loss.backward()
      optimizer.step()

    model.eval()

    val_loss = 0.0
    total = 0
    correct = 0
    with torch.no_grad():
      for data in dl_val:
        X_val = data['X'].to(device)
        y_val = data['y'].to(device)

        y_val_pred = model(X_val)

        val_loss += criterion(y_val_pred, y_val).item()

      print(val_loss)

In [39]:
train(train_loader, val_loader, batch_size, model, loss_fn, optimizer, n_epochs, device)

KeyError: ignored